#### Keras Tuner - Decide the Number of Hidden Layers and Neurons in Neural Network 

In [1]:
#!pip install keras-tuner

In [2]:
import pandas as pd 
import numpy as np 
from tensorflow import keras 
from tensorflow.keras import layers 
import keras_tuner
from keras_tuner.tuners import RandomSearch

In [3]:
df = pd.read_csv('Real_Combine.csv')

In [4]:
df.head() 

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


This dataset is mainly about predicting the AQI in a particular area. The last column PM2.5 is the target variable so this is a regression problem

In [5]:
X = df.iloc[:, :-1] #independent features 
y = df.iloc[:, -1] #dependent/target variable

#### Hyperparameters to Tune 
- Number of hidden layers 
- Number of neurons in each hidden layer 
- Learning rate 

In [6]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [7]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')

Reloading Tuner from project\Air Quality Index\tuner0.json


In [8]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [9]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [10]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))


Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
18                |13                |num_layers
96                |256               |units_0
0.001             |0.01              |learning_rate

Epoch 1/5
24/24 [==============================] - 2s 20ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 2/5
24/24 [==============================] - 0s 5ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 3/5
24/24 [==============================] - 0s 6ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 4/5
24/24 [==============================] - 0s 5ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 5/5
24/24 [==============================] - 0s 7ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 1/5
24/24 [=========

C:\Users\Akshay\Downloads\python\lib\site-packages\keras_tuner\src\engine\metrics_tracking.py:111: RuntimeWarning: All-NaN axis encountered
  np.nanmin(values) if self.direction == "min" else np.nanmax(values)


RuntimeError: Number of consecutive failures exceeded the limit of 3.
